# Getting and filtering data from production redis

In [1]:
from urllib.parse import urlparse
from dotenv import load_dotenv
import os
from pathlib import Path
import json
import redis
import pandas as pd # only if needed for filtering of data

In [2]:
# load .env file
env_path = Path('../') / '.env'
load_dotenv(env_path)

True

In [3]:
def get_cic_stats_from_redis(redis_url):
    # Connect to Redis database
    parsed_url = urlparse(redis_url)
    r = redis.Redis(host=parsed_url.hostname,
                    port=parsed_url.port,
                    db=0,
                    password=parsed_url.password,
                    username=parsed_url.username)
    
    # get objects from redis
    redis_objects = r.mget(r.keys(pattern="cic:*CIC*lastStat*"))
    results = []
    for obj, key in zip(redis_objects, r.keys(pattern="cic:*CIC*lastStat*")):
        try:
            data = json.loads(obj.decode())
            data['cic'] = key.decode().split(':')[1]
            results.append(data)
        except:
            pass
    return results

### Example of usage: getting all cic's connected via Wifi

In [4]:
# get cic data from redis
REDIS_URL = os.getenv("REDISPROD")

redis_data = get_cic_stats_from_redis(REDIS_URL)

df_redis = pd.json_normalize(redis_data)

# select list of cic ids which are connected via Lte
cic_ids = df_redis[(df_redis['system.quattBuild'] != "2.0.1")][['system.quattId', 'system.quattBuild','time.tsHuman', 'system.isLteConnected','system.isWifiReachable','system.isEthernetReachable']]

# print list of cic ids
print(cic_ids)

                                system.quattId system.quattBuild  \
0     CIC-90ecb6cd-8b7c-5685-b9d2-13ab8623d275             2.0.5   
2                                         None             2.0.4   
3     CIC-4d5ecd4f-f1c7-5f02-bd6d-ae974808bd6e             1.2.6   
4     CIC-c8a4a4ed-b6c9-549f-8c35-df4e15aa4437             2.0.5   
5     CIC-55fb7e30-0c71-5010-84e9-431b7d8712be             2.0.5   
...                                        ...               ...   
2354  CIC-b48549a4-b773-54c6-a576-84b390a11aa8             2.0.5   
2355  CIC-5d08dfe4-17bb-5a67-9b8d-4ff71cb4143c             2.0.5   
2356  CIC-21d9e8ee-4092-51e2-96d8-125e3a7bb9c8             2.0.5   
2357  CIC-93ab8148-cee2-59b6-a298-c5abe8a90b1e             2.0.5   
2359  CIC-421420d6-8b26-5bc2-8f6f-e15afe281eb3           0.01.03   

                  time.tsHuman system.isLteConnected system.isWifiReachable  \
0     2023-07-25T09:01:45.176Z                  True                   True   
2     2023-07-25T09:01:51

In [10]:
print(df_redis[df_redis['system.menderId'].isna()][['system.quattBuild']].value_counts())
print(df_redis[df_redis['system.menderId'].notna()][['system.quattBuild']].value_counts())

system.quattBuild
2.0.4                594
2.0.3                 10
2.0.5                  5
2.0.1                  3
2.1.0                  1
dtype: int64
system.quattBuild  
2.0.5                  1372
2.0.1                   254
2.0.3                    33
1.2.6                    18
1.2.5                    13
2.0.4                    10
0.1.3                     6
0.0.28                    4
0.01.03                   3
01.02.05-factory          2
0.0.44                    2
0.00.57                   2
2.0.2                     2
01.02.06-production       2
1.1.0                     1
2.0.0                     1
0.00.44                   1
0.0.57                    1
0.0.53                    1
dtype: int64


In [13]:
cic_ids[['system.quattId', 'system.quattBuild','time.tsHuman', 'system.isLteConnected','system.isWifiReachable','system.isEthernetReachable']]